In [ ]:
#| default_exp utils

# utils

> util functions for the ipfspy library

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import json
from fastcore.all import *
import pandas as pd
import dag_cbor
from typing import Union, Dict, Any
import time
import os
import hashlib

In [ ]:
# #export
# GATEWAYS_API_READ = [
#     "https://ipfs.io/api/v0",
#     "https://gateway.pinata.cloud/api/v0",
#     "https://cloudflare-ipfs.com/api/v0",
#     "https://dweb.link/api/v0",
#     "https://ipfs.eth.aragon.network/api/v0",
#     "https://permaweb.eu.org/api/v0",
#     "https://nftstorage.link/api/v0",
#     "https://ipfs.lain.la/api/v0",
#     "https://ipfs.mihir.ch/api/v0",
#     "https://ipfs.telos.miami/api/v0",
#     "https://jorropo.net/api/v0",
#     "https://cf-ipfs.com/api/v0",
#     "https://cloudflare-ipfs.com/api/v0",
#     "https://gateway.ipfs.io/api/v0",
#     "https://infura-ipfs.io/api/v0",
#     "https://via0.com/api/v0",
#     "https://ipfs.azurewebsites.net/api/v0"
# ]

# GATEWAYS_API_WRITE = [
#     "https://ipfs.io/api/v0",
#     "https://gateway.pinata.cloud/api/v0",
#     "https://cloudflare-ipfs.com/api/v0",
#     "https://dweb.link/api/v0",
# ]

In [ ]:
#| export
def parse_response(
    response, # Response object
):
    "Parse response object into JSON"
    
    if response.text.split('\n')[-1] == "":
        try:
            return [json.loads(each) for each in response.text.split('\n')[:-1]]
        
        except:
            pass

    try:
        return response.json()

    except:
        return response.text
    
    
def parse_error_message(
    response, # Response object from requests
):
    'Parse error message for raising exceptions'
 
    sc = response.status_code
    
    try:
        message = response.json()['Message']
        
    except:
        message = response.text
    
    return f"Response Status Code: {sc}; Error Message: {message}"

In [ ]:
#| export
GATEWAY_MAP = {
    'local': [f"http://127.0.0.1:5001"],
    'infura': ['https://ipfs.infura.io:5001'],
    'public': ["https://ipfs.io",
               "https://gateway.pinata.cloud",
               "https://cloudflare-ipfs.com",
               "https://dweb.link"]
}

In [ ]:
#|export
try:
    if os.environ["IPFSHTTP_LOCAL_HOST"] == "ipfs": GATEWAY_MAP['local'] = [f"http://{os.environ['IPFSHTTP_LOCAL_HOST']}:5001"]
except: pass

In [ ]:
#| export
class IPFSGateway:
    def __init__(self, url):
        self.url = url
        self.state = "unknown"
        self.min_backoff = 1e-9
        self.max_backoff = 5
        self.backoff_time = 0
        self.next_request_time = time.monotonic()
        self.session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)

    def get(self, path):
        try:
            res = self.session.get(self.url + "/ipfs/" + path)
        except requests.ConnectionError as e:
            self._backoff()
            return None
        # this is from https://blog.petrzemek.net/2018/04/22/on-incomplete-http-reads-and-the-requests-library-in-python/
        expected_length = res.headers.get('Content-Length')
        if expected_length is not None:
            actual_length = res.raw.tell()
            expected_length = int(expected_length)
            if actual_length < expected_length:
                # if less than the expected amount of data is delivered, just backoff which will will eiter trigger a
                # retry on the same server or will fall back to another server later on.
                self._backoff()
                return None

        if res.status_code == 429:  # too many requests
            self._backoff()
            return None
        elif res.status_code == 200:
            self._speedup()
        res.raise_for_status()
        return res.content

    def head(self, path, headers=None):
        try:
            res = self.session.get(self.url + "/ipfs/" + path)
        except requests.ConnectionError as e:
            self._backoff()
            return None
        if res.status_code == 429:  # too many requests
            self._backoff()
            return None
        elif res.status_code == 200:
            self._speedup()
        res.raise_for_status()
        return res.headers

    def apipost(self, call, **kwargs):
        try:
            data = kwargs.pop('data') if 'data'  in kwargs else {}
            headers = kwargs.pop('headers') if 'headers' in kwargs else {}
            files = kwargs.pop('files') if 'files' in kwargs else {}
            params = kwargs['params'] if 'params' in kwargs else kwargs

            res = self.session.post(self.url + "/api/v0/" + call, 
                                    params=params, 
                                    data=data, 
                                    headers=headers,
                                    files=files)
        
        except requests.ConnectionError:
            self._backoff()
            return None
        
        if res.status_code == 429:  # too many requests
            self._backoff()
            return None
        
        elif res.status_code == 200:
            self._speedup()
        
        return res

    def _schedule_next(self):
        self.next_request_time = time.monotonic() + self.backoff_time

    def _backoff(self):
        self.backoff_time = min(max(self.min_backoff, self.backoff_time) * 2,
                                self.max_backoff)
        self._schedule_next()

    def _speedup(self):
        self.backoff_time = max(self.min_backoff, self.backoff_time * 0.9)
        self._schedule_next()

    def _init_state(self):
        try:
            res = self.session.post(self.url + "/api/v0/version")
            if res.ok:
                self.state = "online"
            else:
                self.state = "offline"
        except requests.ConnectionError:
            self.state = "offline"

    def get_state(self):
        if self.state == "unknown":
            self._init_state()
        now = time.monotonic()
        if self.next_request_time > now:
            return ("backoff", self.next_request_time - now)
        else:
            return (self.state, None)

In [ ]:
#|export
def dict_get(input_dict,keys, default_value=False):
    """
    get keys that are dot seperated (key1.key2.key3) recursively into a dictionary
    """
    if isinstance(keys, str):
        keys = keys.split('.')

    key = keys[0]

    try:

        next_object_list = [input_dict[key]]
        for key in keys[1:]:
            next_object_list += [next_object_list[-1][key]]
        return next_object_list[-1]
    except Exception as e:
        return default_value
    

def dict_put(input_dict,keys, value ):
    """
    insert keys that are dot seperated (key1.key2.key3) recursively into a dictionary
    """
    if isinstance(keys, str):
        keys = keys.split('.')
    key = keys[0]
    if len(keys) == 1:
        assert isinstance(input_dict,dict)
        input_dict[key] = value

    elif len(keys) > 1:
        if key not in input_dict:
            input_dict[key] = {}
        dict_put(input_dict=input_dict[key],
                             keys=keys[1:],
                             value=value)
        

def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    # We need to sort arguments so {'a': 1, 'b': 2} is
    # the same as {'b': 2, 'a': 1}
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()


def dict_equal(*args):

    if not all([ isinstance(arg, dict) for arg in args]):
        return False
    for i in range(len(args)):
        for j in range(len(args)):
            if dict_hash(args[i]) != dict_hash(args[j]):
                return False

    return True

In [ ]:
#|hide
from nbdev.doclinks import *
nbdev_export()